# Stock Price Prediction

## Imports

In [179]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler


## Loading in the stock data

In [103]:
# Scrape S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data = pd.read_html(url)
sp500_tickers = data[0]['Symbol'].tolist()


In [104]:
# Get the current date
end_date = datetime.datetime.now().strftime("%Y-%m-%d")

# start date
start_date = '1990-01-01'

sp500_data = {}

# Loop through each ticker in the S&P 500 list
for ticker in sp500_tickers:
    # Download historical data for the ticker from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Store the data in a Pandas DataFrame with the ticker as the column name
    sp500_data[ticker] = pd.DataFrame(data)




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [105]:
# Print the first few rows of each DataFrame
for ticker, data_frame in sp500_data.items():
    print(f"First few rows of {ticker}:")
    print(data_frame.head())


First few rows of MMM:
                Open      High       Low     Close  Adj Close   Volume
Date                                                                  
1990-01-02  19.84375  20.15625  19.78125  20.12500   7.398872  1496000
1990-01-03  20.15625  20.46875  20.12500  20.34375   7.479299  2631600
1990-01-04  20.21875  20.75000  20.21875  20.50000   7.536744  2538400
1990-01-05  20.40625  20.50000  20.15625  20.15625   7.410361  1825600
1990-01-08  20.12500  20.71875  20.12500  20.68750   7.605675  2164400
First few rows of AOS:
                Open      High       Low     Close  Adj Close  Volume
Date                                                                 
1990-01-02  0.722222  0.722222  0.722222  0.722222   0.351763   18000
1990-01-03  0.715278  0.715278  0.715278  0.715278   0.348381    9000
1990-01-04  0.722222  0.722222  0.722222  0.722222   0.351763  459000
1990-01-05  0.722222  0.722222  0.715278  0.715278   0.348381    3600
1990-01-08  0.715278  0.722222  0.715

## Checking for missing values

In [106]:
# Loop through each ticker in the S&P 500 list
for ticker in sp500_tickers:
    # Download historical data for the ticker from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Store the data in a Pandas DataFrame with the ticker as the column name
    sp500_data[ticker] = pd.DataFrame(data)
    
    # Check for missing values
    missing_values = sp500_data[ticker].isnull().sum()
    
    # missing values for each ticker
    print(f"Missing values for {ticker}:")
    print(missing_values)


[*********************100%%**********************]  1 of 1 completed


Missing values for MMM:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


[*********************100%%**********************]  1 of 1 completed


Missing values for AOS:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

Missing values for YUM:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Missing values for ZBRA:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


[*********************100%%**********************]  1 of 1 completed


Missing values for ZBH:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Missing values for ZION:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Missing values for ZTS:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [107]:
print(sp500_data['AAPL']['Open'])

Date
1990-01-02      0.314732
1990-01-03      0.339286
1990-01-04      0.341518
1990-01-05      0.337054
1990-01-08      0.334821
                 ...    
2024-02-05    188.149994
2024-02-06    186.860001
2024-02-07    190.639999
2024-02-08    189.389999
2024-02-09    188.649994
Name: Open, Length: 8593, dtype: float64


There was no timezone found for 'BKR' and 'BR', hency why we are going to remove the tickers

In [108]:
#['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
#['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1990-01-01 -> 2024-02-11)')

In [163]:
tickers_to_drop = ['BRK', 'BF'] 

for symbol in tickers_to_drop:
    sp500_data.pop(symbol, None)


In [164]:
# Print keys before removal
print("Keys before removal:", sp500_data.keys())

# Remove tickers
tickers_to_drop = ['BRK', 'BF']
for symbol in tickers_to_drop:
    sp500_data.pop(symbol, None)

# Print keys after removal
print("Keys after removal:", sp500_data.keys())


Keys before removal: dict_keys(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CV

In [170]:
ticker_to_check = 'BRK' and 'BF'
print(ticker_to_check in sp500_data.keys())  

False


There are no missing values for any of the stock tickers. We can proceed.

## Feature engineering

First of all, we are going to predict the stock prices by just simply using the available features. This will serve as a baseline for the upcoming improvements that will be applied.

In the following, a target variable column is created.By calculating the difference between the closing and opening prices of a stock, we aim to capture the daily price movement, which serves as the target variable for machine learning algorithms. Shifting the target values by one row ensures that each day's target corresponds to the price movement on the subsequent trading day, facilitating the training of predictive models to forecast future price changes based on historical data.

Furthermore, we are going to use the target column in order to define a new column 'Direction'. The direction column willindicate if the price is going to go up (+1) or down (-1).

In [217]:
for ticker, df in sp500_data.items():
    sp500_data[ticker]['Target'] = sp500_data[ticker]['Adj Close'] - sp500_data[ticker]['Open']
    sp500_data[ticker]['Target'] = sp500_data[ticker]['Target'].shift(-1)
    
    # Add a new column to indicate the direction of price change
    sp500_data[ticker]['Direction'] = np.where(sp500_data[ticker]['Target'] > 0, 1, -1)


In [218]:
print(sp500_data['AAPL']) 

                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
1990-01-02  0.001009  0.001085  0.001009  0.001096   0.000836  0.024369   
1990-01-03  0.001134  0.001108  0.001122  0.001108   0.000845  0.027712   
1990-01-04  0.001145  0.001141  0.001111  0.001113   0.000849  0.029534   
1990-01-05  0.001122  0.001119  0.001100  0.001119   0.000854  0.016298   
1990-01-08  0.001111  0.001108  0.001100  0.001130   0.000863  0.013367   
...              ...       ...       ...       ...        ...       ...   
2024-01-30  0.964225  0.960802  0.951596  0.949140   0.949144  0.007206   
2024-01-31  0.944519  0.937244  0.935750  0.930756   0.930762  0.007153   
2024-02-01  0.929107  0.936492  0.933058  0.943180   0.943185  0.008423   
2024-02-02  0.908239  0.938397  0.909846  0.938079   0.938085  0.013495   
2024-02-05  0.950128  0.948021  0.943317  0.947322   0.947326  0.009067   

              Target  Di

We will use this as a basis for predicting the price movement for the next day.

### Scaling

We are going to scale the data using MinMaxScaler

In [219]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply scaling
for ticker, df in sp500_data.items():
    # Exclude non-numeric columns like Date
    numeric_cols = df.select_dtypes(include='number').columns
    numeric_cols = numeric_cols.drop('Direction')
    sp500_data[ticker][numeric_cols] = scaler.fit_transform(df[numeric_cols])

The last row of the target column which should represent the stock movement that needs to be predicted is nan. We are going to remove that row from each ticker and apply the models

In [220]:
# Check for NaN values in each column
nan_counts = df.isna().sum()

# Print the number of NaN values in each column
print("Number of NaN values in each column:")
print(nan_counts)


Number of NaN values in each column:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
Target       1
Direction    0
dtype: int64


In [221]:
# Iterate over each ticker and its corresponding DataFrame
for ticker, df in sp500_data.items():
    # Drop the last row
    sp500_data[ticker] = df.drop(df.tail(1).index, inplace=False)

In [222]:
print(sp500_data['AAPL']) 

                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
1990-01-02  0.001009  0.001085  0.001009  0.001096   0.000836  0.024369   
1990-01-03  0.001134  0.001108  0.001122  0.001108   0.000845  0.027712   
1990-01-04  0.001145  0.001141  0.001111  0.001113   0.000849  0.029534   
1990-01-05  0.001122  0.001119  0.001100  0.001119   0.000854  0.016298   
1990-01-08  0.001111  0.001108  0.001100  0.001130   0.000863  0.013367   
...              ...       ...       ...       ...        ...       ...   
2024-01-29  0.969632  0.962807  0.962313  0.967777   0.967780  0.006031   
2024-01-30  0.964225  0.960802  0.951596  0.949140   0.949144  0.007206   
2024-01-31  0.944519  0.937244  0.935750  0.930756   0.930762  0.007153   
2024-02-01  0.929107  0.936492  0.933058  0.943180   0.943185  0.008423   
2024-02-02  0.908239  0.938397  0.909846  0.938079   0.938085  0.013495   

              Target  Di

In [204]:
# Iterate over each ticker and its corresponding DataFrame
for ticker, df in sp500_data.items():
    # Separate features and target variables
    X = df.drop(columns=['Target'])
    y = df['Target']
# Check for NaN values in y
    if y.isna().any():
        print(f"NaN values found in the target variable for ticker {ticker}.")
    else:
        print(f"No NaN values found in the target variable for ticker {ticker}.")

No NaN values found in the target variable for ticker MMM.
No NaN values found in the target variable for ticker AOS.
No NaN values found in the target variable for ticker ABT.
No NaN values found in the target variable for ticker ABBV.
No NaN values found in the target variable for ticker ACN.
No NaN values found in the target variable for ticker ADBE.
No NaN values found in the target variable for ticker AMD.
No NaN values found in the target variable for ticker AES.
No NaN values found in the target variable for ticker AFL.
No NaN values found in the target variable for ticker A.
No NaN values found in the target variable for ticker APD.
No NaN values found in the target variable for ticker ABNB.
No NaN values found in the target variable for ticker AKAM.
No NaN values found in the target variable for ticker ALB.
No NaN values found in the target variable for ticker ARE.
No NaN values found in the target variable for ticker ALGN.
No NaN values found in the target variable for ticker

## Linear Regression

In [223]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Dictionary to store the models and evaluation results
models = {}

# Iterate over each ticker and its corresponding DataFrame
for ticker, df in sp500_data.items():
    # Separate features 
    X = df.drop(columns=['Direction']) 
    y = df['Direction']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Initialize Logistic Regression model
    model = LogisticRegression()
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Evaluate the model on the testing data
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store the model and evaluation results
    models[ticker] = {'model': model, 'accuracy': accuracy}

# Access individual model and evaluation results
for ticker, info in models.items():
    print(f"Ticker: {ticker}, Accuracy: {info['accuracy']}")


Ticker: MMM, Accuracy: 0.9809856422196352
Ticker: AOS, Accuracy: 0.9825378346915018
Ticker: ABT, Accuracy: 0.9856422196352348
Ticker: ABBV, Accuracy: 0.9832935560859188
Ticker: ACN, Accuracy: 0.9858989424206815
Ticker: ADBE, Accuracy: 0.6907256499805976
Ticker: AMD, Accuracy: 0.7306946061311602
Ticker: AES, Accuracy: 0.7727272727272727
Ticker: AFL, Accuracy: 0.9906868451688009
Ticker: A, Accuracy: 0.9244663382594417
Ticker: APD, Accuracy: 0.9906868451688009
Ticker: ABNB, Accuracy: 0.8865546218487395
Ticker: AKAM, Accuracy: 0.9143013100436681
Ticker: ALB, Accuracy: 0.9712643678160919
Ticker: ARE, Accuracy: 0.9801488833746899
Ticker: ALGN, Accuracy: 0.7075417386298215
Ticker: ALLE, Accuracy: 0.9390402075226978
Ticker: LNT, Accuracy: 0.9751649204501358
Ticker: ALL, Accuracy: 0.9861890375485541
Ticker: GOOGL, Accuracy: 0.7421768707482993
Ticker: GOOG, Accuracy: 0.7482993197278912
Ticker: MO, Accuracy: 0.9992239037640668
Ticker: AMZN, Accuracy: 0.6853320118929633
Ticker: AMCR, Accuracy: 0.9

## Random Forest Regressor

In [227]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd


models = {}

# Iterate over each ticker and its corresponding DataFrame
for ticker, df in sp500_data.items():
    X = df.drop(columns=['Target'])
    y = df['Target']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Initialize Random Forest model
    model = RandomForestRegressor(n_estimators=10, random_state=42)
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Evaluate the model on the testing data
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # Store the model and evaluation results
    models[ticker] = {'model': model, 'mse': mse}

# Access individual model and evaluation results
for ticker, info in models.items():
    print(f"Ticker: {ticker}, MSE: {info['mse']}")


Ticker: MMM, MSE: 0.0005638929835113737
Ticker: AOS, MSE: 0.0011041356517292951
Ticker: ABT, MSE: 0.001205330322731895
Ticker: ABBV, MSE: 0.0013563118601635476
Ticker: ACN, MSE: 0.002128246461590641
Ticker: ADBE, MSE: 0.0008257513680081157
Ticker: AMD, MSE: 0.0009926396831932114
Ticker: AES, MSE: 0.0007670766740119565
Ticker: AFL, MSE: 0.000590881811684822
Ticker: A, MSE: 0.0008301814840233333
Ticker: APD, MSE: 0.0012168674743585693
Ticker: ABNB, MSE: 0.010498302220567404
Ticker: AKAM, MSE: 0.0006235610163037844
Ticker: ALB, MSE: 0.0013855496855848216
Ticker: ARE, MSE: 0.0006130379364125634
Ticker: ALGN, MSE: 0.0013584445451390122
Ticker: ALLE, MSE: 0.002252447289100174
Ticker: LNT, MSE: 0.00043306527743894703
Ticker: ALL, MSE: 0.0011796805503745788
Ticker: GOOGL, MSE: 0.0020879418691011307
Ticker: GOOG, MSE: 0.0020828991886687885
Ticker: MO, MSE: 5.289523049717093e-05
Ticker: AMZN, MSE: 0.0012426684314751555
Ticker: AMCR, MSE: 0.0006878562885361224
Ticker: AEE, MSE: 0.0004526339889482